In [4]:
import numpy as np
!pip install scikit-learn # Install the correct package

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

import pandas as pd

from collections import Counter
from sklearn.tree import DecisionTreeClassifier as DecisionTree # Import DecisionTreeClassifier and alias it as DecisionTree

In [5]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]


def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

In [14]:
class RandomForest:

    def __init__(self, n_trees=10, min_samples_split=2,
                 max_depth=100, n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats  # Store n_feats as an attribute
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            # Pass max_features instead of n_feats
            tree = DecisionTree(min_samples_split=self.min_samples_split,
                                max_depth=self.max_depth, max_features=self.n_feats)
            X_samp, y_samp = bootstrap_sample(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)

In [7]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [8]:
cols = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness",
        "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]
url = "https://gist.githubusercontent.com/ktisha/c21e73a1bd1700294ef790c56c8aec1f/raw/819b69b5736821ccee93d05b51de0510bea00294/pima-indians-diabetes.csv"
diabetes_data = pd.read_csv(url, skiprows=9, header=None, names=cols)

In [9]:
diabetes_data.shape


(768, 9)

In [10]:
X = diabetes_data[cols[:-1]].values
y = diabetes_data[cols[-1]].values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)


In [15]:
rnd_clf = RandomForest(max_depth=10, n_trees=10, n_feats=6)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)

In [16]:
print(f"Accuracy: {accuracy(y_test, y_pred)}")


Accuracy: 0.7662337662337663


In [17]:
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1-Score: {f1_score(y_test, y_pred)}")

Precision: 0.68
Recall: 0.6296296296296297
F1-Score: 0.6538461538461539
